In [1]:
# instruciton datasetの生成

In [16]:
import os

#dataフォルダ内をリセット
os.system("mkdir data")
os.system("rm -rf data/*")

mkdir: cannot create directory ‘data’: File exists


0

In [17]:
from datasets import load_dataset
import json

In [18]:
ds_dict={}

# 自動生成したQAの読み込み

In [19]:
def clean_autogen(text):
    if text is None:
        return ""
    text=text.strip()
    return text

In [20]:
#original template
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n### 応答:\n"

#custom template
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
answer_template="<SEP>応答<SEP>"


#question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。:### 指示::"
#answer_template=":### 応答::"

In [21]:
#ベンチマークに用いるjmt benchと類似しすぎたデータは使わないようにする
import pandas as pd
#!pip install rapidfuzz
from rapidfuzz.process import cdist

jmt_bench_df=pd.read_csv("reference_data/jmtbench.csv")
bench_questions=jmt_bench_df["問い"].tolist()

def check_jmt_similarity(q,bench_questions):
    return 1  #一度チェックして問題ないなら、チェックしない
    scores = cdist([q], bench_questions,workers=1)
    score=max(scores[0])
    return score

# mixtralで自動生成したQ&A

In [25]:
from tqdm import tqdm
score_threshold=4
sim_threshold=80
records=[]

exclude_count=0


datasets=[
    load_dataset("hatakeyama-llm-team/AutoGeneratedJapaneseQA",split="train"),
    load_dataset("kanhatakeyama/OrcaJaMixtral8x22b",split="train"),

]
for dataset in datasets:
    for original_record in tqdm(iter(dataset)):
        q=clean_autogen(original_record["question"])
        a=clean_autogen(original_record["answer"])
        if q=="" or a=="":
            continue

        if "score" in original_record:
            if original_record["score"] is None:
                continue
            if int(original_record["score"])<score_threshold:
                continue

        if check_jmt_similarity(q,bench_questions)>sim_threshold:
            print("too similar to jmt bench",q)
            continue

        exclude_flag=False

        #回答しないパターンのrecordを除外
        for ng_word in ["申し訳","分からない","分かりません","すみません",
                        "図","表"]:
            if a.find(ng_word)>=0 or q.find(ng_word)>=0:
                #print("excluded:",a)
                exclude_flag=True
                exclude_count+=1
                continue

        if exclude_flag:
            continue
        #if len(a)<10:
        #    print("too short answer",a)
        #    continue

        text=f"{question_template}{q}{answer_template}{a}"
        if a!="":
            records.append(text)


ds_dict["auto_gen_mixtral"]=records
len(records),exclude_count

438565it [00:14, 29659.14it/s]
19524it [00:00, 21532.59it/s]


(179009, 75674)

# hachiさんのalpaca + mixtral dataset

In [26]:
hachi_alpaca=load_dataset("HachiML/Hachi-Alpaca",split='v1.0_cleaned')

Generating _archive_v0.2 split: 100%|██████████| 4003/4003 [00:00<00:00, 136951.14 examples/s]


In [27]:
records=[]
for record in tqdm(hachi_alpaca):
    q=record["instruction"]
    inp=record["input"]
    if inp!="":
        q+="\n"+inp
    a=record["output"]
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
    

ds_dict["hachi_alpaca"]=records

100%|██████████| 28861/28861 [00:03<00:00, 8925.33it/s] 


# Bumpo dataset

In [28]:
#文法理解に関するデータセット
ds2=load_dataset("hatakeyama-llm-team/BumpoRikai",split="train")
ds2

Dataset({
    features: ['instruction', 'question', 'answer'],
    num_rows: 26448
})

In [29]:
records=[]
for original_record in iter(ds2):
    q=(original_record["question"])
    a=(original_record["answer"])
    inst=(original_record["instruction"])
    text=f"{question_template}{q}{answer_template}{a}"
    records.append(text)
print(text)
ds_dict["bumpo_rikai"]=records
records[1]

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>問題　樹液からはチューインガムの原料が採取できる、中南米原産のアカテツ科の植物は何でしょう?<SEP>応答<SEP>サポジラ


'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>faカップ準決勝に延長戦はありますか<SEP>応答<SEP>YES'

# minnade dataset

In [30]:
from datasets import load_dataset
#todo: dataを入れる
m_ds=load_dataset("minnade/chat-daily",split="train")

id_to_content={}
for record in m_ds:
    id_to_content[record["id"]]=record["body"]

questions=[]
for record in m_ds:
    if record["role"]=="assistant":
        q=id_to_content[record["parent_id"]]
        a=record["body"]
        questions.append((q,a))
        text=f"{question_template}{q}{answer_template}{a}"

ds_dict["minnade"]=questions

In [32]:
all_recrds=[]
for k,v in ds_dict.items():
    all_recrds+=v

len(all_recrds)

234626

In [16]:
import random

def write_jsonl(records,
    output_path="data/all.jsonl",
    n_eval=500,
    n_train=10**7,
    ):

    random.shuffle(records)

    lines=[json.dumps({"text":text},ensure_ascii=False) for text in records]
    with open (output_path,"w", newline='\n') as f:
        temp_lines=lines[:-n_eval][:n_train]
        obj = map(lambda x: x + "\n", temp_lines)
        f.writelines(obj)

    lines=[json.dumps({"text":text},ensure_ascii=False) for text in records]
    lines=lines[-n_eval:]
    obj = map(lambda x: x + "\n", lines)
    with open (output_path+".eval","w", newline='\n') as f:
        f.writelines(obj)

n_train=10**7
#n_train=1000
write_jsonl(all_recrds,"data/all.jsonl",n_train=n_train)